In [1]:
import os 
import glob
from pathlib import Path
import pandas as pd 
import numpy as np 
import ast
import json 

In [2]:
map_dti = pd.read_csv('/working/lupolab/julia/tcia_analysis/code/dti_seg_map.csv', index_col = 0)

In [3]:
map_dti.head()

,cohort,dti,exam,patient_id,segmentation_name
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,6-AXIAL_FLAIR-84159
1,TCGA-LGG-nifti,"['205-dadc-52244.nii.gz', '205-dadc-52244a.nii...",06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,401-T2_AX_FLAIR-72445
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz', '...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,17-FLAIR_axial-36217
3,TCGA-LGG-nifti,"['901-PROP_eADC-45332.nii.gz', '900-PROP_ADC-5...",08-16-1995-MRI_BRAIN_WWO_CONTRAST-29908,TCGA-HT-7684,7-Ax_Flair_irFSE_H-26694
4,TCGA-LGG-nifti,"['601-DTI_ASSET-84919a.nii.gz', '601-DTI_ASSET...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,8-AXIAL_FLAIR-42112


## Here we're going to figure out which came with preprocessed ADC: 

In [4]:
map_dti['adc'] = ['' for x in range(map_dti.shape[0])]

In [5]:
for idx, row in map_dti.iterrows(): 
    list_of_dti = ast.literal_eval(row['dti'])
    adc_list = [x for x in list_of_dti if 'adc' in x.lower() or 'apparent' in x.lower() or 'coeff' in x.lower() or 'average' in x.lower()]
    if adc_list: 
        row['adc'] = adc_list
    else: 
        row['adc'] = float("NaN")

In [6]:
map_dti['adc'].isnull().value_counts()

False    101
True      54
Name: adc, dtype: int64

In [7]:
map_dti.head()

,cohort,dti,exam,patient_id,segmentation_name,adc
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,6-AXIAL_FLAIR-84159,[TCGA-DU-5853_adc.nii.gz]
1,TCGA-LGG-nifti,"['205-dadc-52244.nii.gz', '205-dadc-52244a.nii...",06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,401-T2_AX_FLAIR-72445,"[205-dadc-52244.nii.gz, 205-dadc-52244a.nii.gz]"
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz', '...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,17-FLAIR_axial-36217,"[12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz, 10-..."
3,TCGA-LGG-nifti,"['901-PROP_eADC-45332.nii.gz', '900-PROP_ADC-5...",08-16-1995-MRI_BRAIN_WWO_CONTRAST-29908,TCGA-HT-7684,7-Ax_Flair_irFSE_H-26694,"[901-PROP_eADC-45332.nii.gz, 900-PROP_ADC-5240..."
4,TCGA-LGG-nifti,"['601-DTI_ASSET-84919a.nii.gz', '601-DTI_ASSET...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,8-AXIAL_FLAIR-42112,[TCGA-DU-7015_adc.nii.gz]


## Here are the manually processed DTI: 

In [8]:
for idx, row in map_dti.iterrows(): 
    
    path = '/working/lupolab/julia/tcia_analysis/datasets/'+'-'.join(row['cohort'].split('-')[:-1])+'/'+row['patient_id']+'/'+row['exam']
    series_in_exam = os.listdir(path)
    print(series_in_exam)
    if 'diffusion_b=1000' in series_in_exam : 
        row['adc'] = 'diffusion_b=1000'
    elif 'diffusion_b=1200' in series_in_exam: 
        row['adc'] = 'diffusion_b=1200'
    

['3-SAG_T1_FLAIR-94337', '4-DTI_2.6mm_WH-81596', '12-COR__T1_POST_GD_FLAIR-32689', '401-DTI_2.6mm_WH-97906', '7-AX_T2_FR-FSE_RF2_150-55295', '10-3D_DCE_DYNAMIC_PERF-61302', '11-AX_T1_POST_GD_FLAIR-81300', '6-AXIAL_FLAIR-84159', '5-AXIAL_DIFFUSION-35619', 'diffusion_b=1000', '8-AX_T1_pre_gd-14019', '400-DTI_2.6mm_WH-19253', '9-3D_DCE_T1_MAP-73753', '402-DTI_2.6mm_WH-69647']
['1101-STRYKER-27064', '605-MRA-59702', '701-AX_PD-23493', '201-DIFFUSION-10198', '301-T1_SAG_SE-41348', '101-SurveyMST-18717', '1201-T1_COR_POST-35170', '801-T2WFFE_bleed-67839', '1301-T1_SAG_POST-08017', '603-RT___MRA-44425', '602-3D_MC__MRA-83030', '901-T1_SE_PRE-16483', '401-T2_AX_FLAIR-72445', '501-T2-WHOLE_BRAIN-32557', '203-eB_0-99844', '205-dadc-52244', '204-eB1000I-81870', '601-3D_MC__MRA-29688', '202-Reg_-_DIFFUSION_SENSE-00561', '1001-T1_SE_POST-94212', '604-LT___MRA-27206']
['13-ep2ddiffmddw20p2_wipTRACEW-63745', '11-ep2ddiffmddw20p2_wip-78279', '15-ep2ddiffmddw20p2_wipColFA-13380', '19-T1mprageAx_Gd-8358

['15-SAG_T1_SEC-28402', '1401-Apparent_Diffusion_Coefficient_mms-12637', '14-2D_EPI_DWI_ASSET-50533', '8-T2Prop-97284', '501-PROP_eADC-44553', '7-Ax_T2_Flair-25684', '500-PROP_ADC-33609', '13-Ax_T1_MP_SPGR_C-64481', '11-COR_T1_SEC-12641', '4-Sag_T2_FSE-92782', '1400-Exponential_Apparent_Diffusion_Coefficient-13771', '10-Cor_2D_T2GRE-76525', '9-Ax_T1_SE-48997', '502-PROP_COMB-42349', '6-OPTIONALAx_T2_FSE_INTER-50083', '5-8hrbrainAx2DZoomDWIProp-92850']
['4-DTI_2.6mm_WH-00936', '10-AX_T1_POST_GD_FLAIR-74207', '7-AX_T2_FR-FSE_RF2_150-02988', 'diffusion_b=1000', '11-COR__T1_POST_GD_FLAIR-50801', '1-3_PLANE_LOC-44485', '6-AXIAL_FLAIR-25902', '5-AXIAL_DIFFUSION-32196', '9-AXIAL_PERFUSION-04494', '3-SAG_T1_FLAIR-32472', '402-DTI_2.6mm_WH-66929', '2-ASSET_cal-29345', '400-DTI_2.6mm_WH-39403', '401-DTI_2.6mm_WH-56261', '8-AX_T1_pre_gd-36450']
['10-3D_DCE_T1_MAPPING_X_5-75063', '2-3_PLANE_LOC-07157', 'diffusion_b=1000', '12-AX_T1_POST_GD_FLAIR-53586', '13-COR__T1_POST_GD_FLAIR-06706', '5-DTI_2.6

['4-AX_3D_SPGR-08228', '8-Ax_Flair_irFSE_H-30437', '901-Apparent_Diffusion_Coefficient_ms-81614', '5-COR_GRE_T2-36967', '1-3-pl_T2_FGRE_S-04369', '11-SAG_T1C-50563', '6-SAG_T1-72398', '9-Ax_DW_DW-EPI_S_b1000T2-84695', '10-AX_3D_SPGRC-88568', '900-Exponential_Apparent_Diffusion_Coefficient-01613', '7-Ax_T2_FSE_H-78773', '12-COR_T1C-08484']
['501-DIFFUSION-08793', '1201-T1_COR_SE-17084', '1003-SAG_MPR_ISO1MM-83617', '401-SAG_T1-02443', '1001-T1W3DSTRYKER-84137', '502-eBo-00599', '1301-T1_SAG_SE-08339', '601-T2WFFE-90138', '101-Survey-94672', '1002-COR_MPR_ISO1MM-45390', '503-eB1000i-73091', '801-AXIAL__T2-43433', '504-dADC_MAP-93398', '1101-T1_AX_SE_FS_POST-08997', '701-T2_AX_FLAIR-08756', '901-T1_AX__SE-78879']
['12-COR_T1C-68868', '802-PROP_COMB-11163', '801-PROP_eADC-93184', '400-Exponential_Apparent_Diffusion_Coefficient-95285', '401-Apparent_Diffusion_Coefficient_mms-85810', '3-SAG_T1-99851', '9-Prop_T2_TRF-28138', '4-Ax_DW_DW-EPI_S_b1000T2-41290', '800-PROP_ADC-80124', '7-COR_GRE_T

['7-ASSET_Calib-69560', '2-AX_DWI-74272', '800-DTI_N27-26260', '10-AXIAL_T1_C-24985', '9-COR_3D_POST-09215', '11-SE_SAG_T1_POST-28373', '12-FSPGR_3D-67548', '13-T2_wand-75589', '3-FSE_T2-71981', '801-DTI_N27-40119', '4-T2_FLAIR-05248', '802-DTI_N27-19564', '6-Axial_T2-64258', '5-SE_T1-06105', '1-3-P_Loc-86486', '8-DTI_N27-04978']
['7-AxFLAIR-thin_for_surgery-06923', '12-COR_T1-73293', '369-Apparent_Diffusion_Coefficien-89403', '11-AxT1-thin_for_surgery-54106', '21-nordicICE_HFH_-_rBF_map_-Leakage_corrected-125.0', '22-nordicICE_HFH_-_MTT_map_-Leakage_corrected-750.0', '10-AXIAL_PERFUSION-13032', '9-AXIAL_DIFFUSION-08121', '3-3_PLANE_LOC-50108', '2-SAG_T1-00552', '5-AXIAL_FSE-67200', '1-3_PLANE_LOC-21963', '20-nordicICE_HFH_-_rBV_map_-Leakage_corrected-219.0', '6-AXIAL_T1-84562', '8-SAG_T1-27364', '4-SAG_T1-90421']
['8-AX_T1_PostFIL-73586', '10-AX_T2_FS-03857', '7-AX_T2_FS-39199', '13-AX_DWI_WIP-91895', '12-SAG_MP-RAGE_Post-30819', '14-AX_DWI_WIPADC-88397', '11-AX_FLAIR-01197', '100-COR

['10-cor_t1_c-64838', '8-ax_diffADC-61763', '4-ax_t2-23490', '5-ax_flair-26552', '9-ax_t1_c-28600', '6-ax_pd-60758', '7-ax_diff-69090', '2-sag_t2-22184', '3-ax_t1-40219', '1-loc-58271']
['1001-T1_SAG_POST-76764', '701-T1_AX_PRE-59877', '301-FLAIR-98130', '502-SS_DWI-31465', '201-T1_SAG-12176', '901-T1_COR_POST-88195', '801-T1_AX_POST-02808', '501-SS_DWI-09953', '601-HEMOSIDERIN-53378', '401-T2_GRASE-38152']
['22-nordicICE_HFH_-_MTT_map_-Leakage_corrected-405.0', '20-nordicICE_HFH_-_rBV_map_-Leakage_corrected-380.0', '12-AxT1-thin_for_surgery-42560', '6-AXIAL_FSE-22516', '7-AxFLAIR-thin_for_surgery-83857', '10-AXIAL_PERFUSION-09480', '172-Apparent_Diffusion_Coefficien-07376', '13-COR_T1-08791', '21-nordicICE_HFH_-_rBF_map_-Leakage_corrected-937.0', '11-AXIAL_PERFUSION-02140', '8-AXIAL_DIFFUSION-27805', '9-AXIAL_T1-39993', '5-SAG_T1-01348']
['3-FLAIR_AXIALS-52753', 'Logfile', '4-TSE_T2_AXIALS-55149', '5-DTI_DIFF-20269', '1-SCOUT-56618', '6-MP_RAGE_AXIAL-14826', 'diffusion_b=', '2-MP_RAGE

['501-BRAIN______SS_DWI-81030', '301-BRAIN______FLAIR-27209', '601-BRAIN______HEMOSIDERIN-45408', '201-BRAIN______T1_SAG-56811', '701-BRAIN______HEMOSIDERIN-98581', '801-BRAIN______T1_AX_PRE-24658', '1101-BRAIN______T1_SAG_POST-59138', '502-BRAIN______SS_DWI-29643', '1001-BRAIN______T1_COR_POST-23785', '401-BRAIN______T2_GRASE-02583', '901-BRAIN______T1_AX_POST-47275']
['13-Axial_T2-47060', '7-FSPGR_3D-56726', 'Logfile', '16-DTI_N27-89084', '2-fMRI_Anat-43638', '1601-DTI_N27-57108', '1-3-P_Loc-31259', '14-FSPGR_3D-90164', '8-T2_FLAIR-2MM-74210', '3-fMRI_-_FAS-49780', '10-FSE_T2-35736', '11-T2_FLAIR-76096', '9-3-P_Loc-22599', '5-fMRI_-_PCOMP-47901', '1600-DTI_N27-36577', '17-Ax_SE_T1_Post-08723', '15-ASSET_Calib-52589', '1602-DTI_N27-33542', '18-POST-Cor_SE_T1-93708', '4-fMRI_-_CAT-77996', '20-FSPGR_3D-14397', '6-fMRI_-_RHand-81987', '19-POST-Sag_SE_T1-83426', 'diffusion_temp', '12-SE_T1-12267']
['10-AXIAL_PERFUSION-81080', '7-AXIAL_T1-48379', '4-AxFLAIR-thin_for_surgery-17004', '5-AXIA

In [15]:
map_dti.adc.isnull().value_counts()

False    101
True      54
Name: adc, dtype: int64

In [16]:
map_dti_to_align = map_dti.loc[~map_dti.adc.isnull()]
map_dti_to_align.shape

(101, 6)

In [17]:
map_dti_to_align.head(10)

,cohort,dti,exam,patient_id,segmentation_name,adc
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,6-AXIAL_FLAIR-84159,diffusion_b=1000
1,TCGA-LGG-nifti,"['205-dadc-52244.nii.gz', '205-dadc-52244a.nii...",06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,401-T2_AX_FLAIR-72445,"[205-dadc-52244.nii.gz, 205-dadc-52244a.nii.gz]"
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz', '...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,17-FLAIR_axial-36217,"[12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz, 10-..."
3,TCGA-LGG-nifti,"['901-PROP_eADC-45332.nii.gz', '900-PROP_ADC-5...",08-16-1995-MRI_BRAIN_WWO_CONTRAST-29908,TCGA-HT-7684,7-Ax_Flair_irFSE_H-26694,"[901-PROP_eADC-45332.nii.gz, 900-PROP_ADC-5240..."
4,TCGA-LGG-nifti,"['601-DTI_ASSET-84919a.nii.gz', '601-DTI_ASSET...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,8-AXIAL_FLAIR-42112,diffusion_b=1000
6,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '500-DTI_2.6...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,4-AXIAL_FLAIR-03012,diffusion_b=1000
7,TCGA-LGG-nifti,['600-Exponential_Apparent_Diffusion_Coefficie...,07-26-1997-MRI_BRAIN_WWO_CONTRAST-17402,TCGA-DU-A5TR,5-AXIAL_FLAIR-79506,[600-Exponential_Apparent_Diffusion_Coefficien...
8,TCGA-LGG-nifti,"['900-PROP_ADC-74032a.nii.gz', '9-Prop_DWI-265...",10-28-1995-MRI_BRAIN_FOR_STEREOTACTIC_WWO_CONT...,TCGA-HT-7604,7-Ax_Flair_irFSE_H-59891,"[900-PROP_ADC-74032a.nii.gz, 900-PROP_ADC-7403..."
9,TCGA-LGG-nifti,['300-Exponential_Apparent_Diffusion_Coefficie...,03-12-1996-MRI_BRAIN_FOR_STEREOTACTIC_WWO_CONT...,TCGA-HT-7690,7-Ax_Flair_irFSE_H-88757,[300-Exponential_Apparent_Diffusion_Coefficien...
12,TCGA-LGG-nifti,"['6-DTI_2.6mm_WH-60904.nii.gz', '602-DTI_2.6mm...",11-19-1996-MRI_BRAIN_WWO_CONTRAST-27038,TCGA-DU-8163,8-AXIAL_FLAIR-03695,diffusion_b=1000


In [119]:
# map_dti_to_align.to_csv('/working/lupolab/julia/tcia_analysis/code/aligned_dti.csv', index = False)

## Register the DTI 

In [12]:
os.chdir('/working/lupolab/julia/tcia_analysis/code')

In [13]:
usable_patient_gbm = pd.read_csv('usable_patient_gbm.csv', index_col = 0)

In [14]:
usable_patient_lgg = pd.read_csv('usable_patient_lgg.csv', index_col = 0)

In [72]:
usable_patients = pd.concat([usable_patient_gbm, usable_patient_lgg], ignore_index=True)

In [76]:
usable_patients.head(1)

,ground_truth,image,sd,patient_id,exam_id,fixed_vol
0,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,701-BRAIN_WWO_T1_COR_POST-08207,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488,/working/lupolab/julia/tcia_analysis/datasets/...


In [18]:
PATH = Path('/working/lupolab/julia/tcia_analysis/datasets/')

In [20]:
usable_patients = pd.read_csv(Path(PATH, 'one_exam_per_patient_with_fixed_vol.csv'))

In [21]:
usable_patients.head(1)

,patient_id,cohort,patient_exam,image,contrast,contrast_label,plane,exam_eligible,fixed_vol
0,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,9-AxT1-thin_for_surgery-27598,1,T1C,ax,1,6-AXIAL_T1-20532


Need to find the fixed volume using patient id/ exam/ & then just create command to do this with diffusion: 

`BRAINSFit --fixedVolume $fixed_vol.nii.gz --movingVolume $moving_vol.nii.gz --outputVolume $output_vol.nii.gz --outputTransform $transform_output.tfm  --transformType Rigid`

In [22]:
map_dti_to_align['aligned_dti'] = ['' for x in range(map_dti_to_align.shape[0])]

/data/svcf/software/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
map_dti_to_align.head()

,cohort,dti,exam,patient_id,segmentation_name,adc,aligned_dti
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,6-AXIAL_FLAIR-84159,diffusion_b=1000,
1,TCGA-LGG-nifti,"['205-dadc-52244.nii.gz', '205-dadc-52244a.nii...",06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,401-T2_AX_FLAIR-72445,"[205-dadc-52244.nii.gz, 205-dadc-52244a.nii.gz]",
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz', '...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,17-FLAIR_axial-36217,"[12-ep2ddiffmddw20p2_wipADC-35015a.nii.gz, 10-...",
3,TCGA-LGG-nifti,"['901-PROP_eADC-45332.nii.gz', '900-PROP_ADC-5...",08-16-1995-MRI_BRAIN_WWO_CONTRAST-29908,TCGA-HT-7684,7-Ax_Flair_irFSE_H-26694,"[901-PROP_eADC-45332.nii.gz, 900-PROP_ADC-5240...",
4,TCGA-LGG-nifti,"['601-DTI_ASSET-84919a.nii.gz', '601-DTI_ASSET...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,8-AXIAL_FLAIR-42112,diffusion_b=1000,


In [32]:
commands = []
for idx, row in map_dti_to_align.iterrows(): 
    patient_id = row['patient_id']
    fixed_vol_df = usable_patients.loc[usable_patients.patient_id == patient_id]['fixed_vol'].reset_index()
    
    fixed_vol = (fixed_vol_df['fixed_vol'][0])
    root_dir = '/working/lupolab/julia/tcia_analysis/datasets/'
    cohort = row['cohort']
    exam = row['exam']
    fixed_vol_with_path = Path(root_dir, cohort, patient_id, exam, fixed_vol+'a.nii.gz')
    if 'diffu' in row['adc']: 
        moving_vol = str(Path(root_dir, cohort, patient_id, exam, patient_id+'_adc.nii.gz'))
        output_vol = str(Path(root_dir, cohort, patient_id, exam, patient_id+'_adca.nii.gz'))
        command_align = 'BRAINSFit --fixedVolume '+str(fixed_vol_with_path)+' --movingVolume '+moving_vol+' --outputVolume '+output_vol+' --transformType Rigid'
        commands.append(command_align)
        row['aligned_dti'] = output_vol
    else: 
#         print(type(row['adc']))
#         list_adc = ast.literal_eval(row['adc'])
        output_vols = []
        for series in row['adc']: 
            moving_vol = str(Path(root_dir, cohort, patient_id, exam, series))
            output_vol = str(Path(root_dir, cohort, patient_id, exam, '.'.join(series.split('.')[:-2])+'a.nii.gz'))
            command_align = command_align = 'BRAINSFit --fixedVolume '+fixed_vol+' --movingVolume '+moving_vol+' --outputVolume '+output_vol+' --transformType Rigid'
            commands.append(command_align)
            output_vols.append(output_vol)
        row['aligned_dti'] = output_vols

In [33]:
commands

['BRAINSFit --fixedVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/8-AX_T1_pre_gd-14019a.nii.gz --movingVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/TCGA-DU-5853_adc.nii.gz --outputVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-5853/08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436/TCGA-DU-5853_adca.nii.gz --transformType Rigid',
 'BRAINSFit --fixedVolume 301-T1_SAG_SE-41348 --movingVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-CS-6186/06-01-2000-MRI_BRAIN_per_R-97608/205-dadc-52244.nii.gz --outputVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-CS-6186/06-01-2000-MRI_BRAIN_per_R-97608/205-dadc-52244a.nii.gz --transformType Rigid',
 'BRAINSFit --fixedVolume 301-T1_SAG_SE-41348 --movingVolume /working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-CS-6186/06-01-

In [34]:
os.chdir('/working/lupolab/julia/tcia_analysis/code/')

In [35]:
x = open('register_ADCs.txt', 'w')

In [36]:
x.write('{')
x.write('\n')
for command in commands: 
    x.write(command)
    x.write('\n')
x.write('}')
x.close()

In [38]:
!grid_pipeline --scriptonly register_ADCs.txt


In [109]:
# Run qsub register_ADC.txt_1.csh on Atom at Sun 3:33 pm Mar 1, 2020
